In [18]:
import pandas as pd 
import numpy as np

In [2]:
contacts = pd.read_json('contacts.json')

In [3]:
contacts.head()

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,,1,
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,,9125983679,0,
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,,300364407,2,


In [4]:
email = pd.DataFrame(contacts[['Id', 'Email']])
email.replace('', np.nan, inplace=True)
email.dropna(subset=['Email'], inplace=True)
email_nice = []
for k, v in email.groupby('Email'):
    email_nice.append(list(v['Id']))

In [5]:
phone = pd.DataFrame(contacts[['Id', 'Phone']])
phone.replace('', np.nan, inplace=True)
phone.dropna(subset=['Phone'], inplace=True)
phone_nice = []
for k, v in phone.groupby('Phone'):
    phone_nice.append(list(v['Id']))

In [6]:
orderId = pd.DataFrame(contacts[['Id', 'OrderId']])
orderId.replace('', np.nan, inplace=True)
orderId.dropna(subset=['OrderId'], inplace=True)
orderId_nice = []
for k, v in orderId.groupby('OrderId'):
    orderId_nice.append(list(v['Id']))

In [7]:
link = email_nice + phone_nice + orderId_nice

In [8]:
import networkx 
from networkx.algorithms.components.connected import connected_components


def to_graph(l):
    G = networkx.Graph()
    for part in l:
        # each sublist is a bunch of nodes
        G.add_nodes_from(part)
        # it also imlies a number of edges:
        G.add_edges_from(to_edges(part))
    return G

def to_edges(l):
    """ 
        treat `l` as a Graph and returns it's edges 
        to_edges(['a','b','c','d']) -> [(a,b), (b,c),(c,d)]
    """
    it = iter(l)
    last = next(it)

    for current in it:
        yield last, current
        last = current    

G = to_graph(link)
print(connected_components(G))

<generator object connected_components at 0x0000022DFC175430>


In [34]:
total_link = []
for i in connected_components(G):
    peep = list(i)
    peep.sort()
    total_link.append(peep)

In [35]:
len(total_link)

291904

In [36]:
ans = {}
for i in total_link:
    sum = 0
    trace = ''
    sum = contacts.iloc[i]['Contacts'].sum()
    for j in i:
        trace += '{0}-'.format(j)
    trace = trace[:-1]
    trace += ', {0}'.format(sum)
    for j in i:
        ans[j] = trace

In [37]:
answer = pd.DataFrame.from_dict(ans, orient = 'index')

In [38]:
answer.head()

,0
112387,"112387-470075, 4"
470075,"112387-470075, 4"
338154,"338154, 2"
426495,"426495, 2"
37868,"37868-126317-308856, 7"


In [42]:
answer = answer.sort_index()

In [40]:
answer.columns = ['ticket_trace/contact']
answer.index.name = 'ticket_id'

In [43]:
answer.head()

,ticket_trace/contact
ticket_id,
0,"0, 1"
1,"1-2458-98519-115061-140081-165605-476346, 12"
2,"2-159312-322639-348955, 4"
3,"3, 0"
4,"4, 2"


In [45]:
answer.to_csv(r'C:\Users\vuduc\Desktop\ans.csv')